In [1]:
import sys

from pymongo import MongoClient

sys.path.append('../config/')
from config import config

In [2]:
params = config(section='mongodb')
client = MongoClient(params["host"], int(params["port"]))
db = client[params["database"]]
table = db.table

In [3]:
table.count()

27384

In [4]:
table.find_one()

{'_id': ObjectId('5afc3e58672caf0c3490a6e3'),
 'region': {'id': '1100', 'name': 'Российская Федерация'},
 'year': 2016,
 'group': {'id': 14002, 'name': 'по возрасту'},
 'category': '14-15',
 'gender': 'Мужчины',
 'value': 13573}

In [5]:
table.count({'year': 2016})

3948

In [6]:
stargroup = table.aggregate([
    {
        '$group': {
            '_id': '$year',
            'count': {'$sum': 1}
        }
    },
    {
        '$sort': {'_id': 1}
    }
])

for group in stargroup:
    print(group)

{'_id': 2010, 'count': 3864}
{'_id': 2011, 'count': 3864}
{'_id': 2012, 'count': 3864}
{'_id': 2013, 'count': 3864}
{'_id': 2014, 'count': 3990}
{'_id': 2015, 'count': 3990}
{'_id': 2016, 'count': 3948}


In [7]:
table.find({'group.id': 14002}).count()

7824

In [8]:
stargroup = table.aggregate([
    {
        '$group': {
            '_id': '$group.id',
            'count': {'$sum': 1}
        }
    },
    {
        '$sort': {'_id': 1}
    }
])

for group in stargroup:
    print(group)

{'_id': 14002, 'count': 7824}
{'_id': 14005, 'count': 5216}
{'_id': 14006, 'count': 11736}
{'_id': 14007, 'count': 2608}


In [9]:
client.close()